# <span style="color:blue"> TEAM SOUTHERN TECHNICAL REPORT </span>

By Richard Boyne, Yusuf Falola, Tayfun Karaderi, Deborah Pelacani Cruz, Deirdrée Polak

# Table of Content

## 1. Solution Algorithm

## 2. Demonstration of Functionality

## 3. Sustainability

## 4. Wave Crest Study

## 5. Artificial Pressure Study

## 6. Further Improvements


# <span style="color:blue"> 1. Solution Algorithm </span>


## Timestepping Algorithms

At each time step, ten particles attribute need to be updated along with the time step itself. The atrributes are:
- particle neighbours, `adj list`
- particles acceleration, or change in velocity, `a_i`
- particles change in density, `D_i`
- particles smoothing, `dW_i`
- particles relative distance, `r_ij`
- particles relative velocity to the neighbours, `v_ij`
- particles pressure, `P`

and the three attributes calculated using the Forward euler or Improved Euler scheme:
- particle's position, `x_i`
- particle's velocity, `v_i`
- particle's dencsity, `rho_i`


The following section will introduce how Forward Euler and Predictor Corrector (also called Improved Euler) method are implementing these changes through the system. 

### a. Forward Euler

The initial implementation of the SPH simulator uses a Forward Euler timestepping scheme. It timesteps the particle's position, velocity and density. This timestepping method is used in `sph_fe`. 

$$
{x}_{i}^{t+1} = {x}_{i}^{t} + {\Delta t}{v}_{i}^{t}\\\\
{v}_{i}^{t+1} = {v}_{i}^{t} + {\Delta t}{a}_{i}^{t}\\\\
{\rho}_{i}^{t+1} = {\rho}_{i}^{t} + {\Delta t}{D}_{i}^{t}\\\\
$$


The flow chart below is a visual representation of the different operations takinf place during each timestep of the SPH simulator. 

<img src="./report_diagrams/fwd_euler.png" width=500x>
    
The Forward Euler gives promising results and allows for a good visualisation of the flow's behaviour. It however require very small time step in order to work and is only first order accurate. 

Optimisation of the `finding neighbours` method was implemented in `sph_fe`


### b. Predictor-Corrector Method

The second and improved implemtation of the SPH simulator makes use of an Imporoved Euler time stepping schemes. This time stepping solution is second order accurate, and is used in both file `sph_ie` and `sph_ap`. The `sph_ap` module has additional methods, to implement the artificial pressure on the SPH simulator. 

The Predictor Corrector method takes the form: 

<img src="./report_diagrams/imp_euler.png" width=500x>

The main challenge of implementing this method was saving the temporary value for particle i and j. While different lists were cosidered, the best implementation was to add `x_temp`, `v_temp` and `rho_temp` as attributes of the particles. 

These additions included adding a few changes to the main code, such as adding a `step` parameter to the `dW` function. This is because in the second step, when `a`, `D`, `dW` and LJ forces are calculated in the double for loop, the value of x that needs to be considered for `r_ij` and `v_ij` is that `x_temp`. 

This implementation worked easily and required little changes to the main code. 

For further improvements to the Predictor Corrector method, the two for loops should be made into methods that are called for the two stages of the PC timestepping. 

## Find Neighbours

A find_neighbours was given and used for early simulations. This method worked well, however as the largest time restraint on this simulation is looping over all neighbours for all particles we should look to optimise the code here. As the derivative equations are either symmetric (density) or anti-symmetric (velocity) we can consider the interaction of particle i on particle j abd the interaction of particle j on particle i at the same time. This halves the nuber of particles we have to iterate over and improves run time by $\sim40-50\%$.

The new method developped here, is implemented by only considering half of the particles neightbours, requiering a new find_neighbours function where only the highlighted grid points below are considered.

<img src="./report_diagrams/l_shape.png" width=200x>



we also need to consider the centre grid cell neighbours, however we must be careful to only consider each particle pair once (i.e. we need an asymetrical criteria so that we include the interaction when looking at particle i but not when looking at particle j). The best way to gaurentee this is used the particles ID numbers: 

```python
if ID_i > ID_j:
    consider interaction
```

This is computationally cheap (they are only ever relativly small integers) and ensures the interactions will never be considered twice. In terms of equations whenever we add one of the summations terms to the accleration of particle i we also subtract it from particle j and we add the density term to both i and j as well. The only other algorithmic change this requiers is to reset the deriatives for all particles at the end of a timestep so we can add sumattion terms for any particle in the next step before they are considered.


## Grid Initialisation

The grid allocation was immproved to suit better with a front end code. The new gird initialisation works by running a series of methods in the main class. This new method also ensures that there are no overlapping particles.

The user input to create the grid consists of 
- x_max, upper corner of the domain
- x_min, lower corner of the domain
- f(x, y), which gives coordinates a value of 1 when a particle is present and 0 when the spaceis empty.

The f(x, y) function is written as inequalities for the case example, such as: 

```python
 def f(x, y):
    if 0 <= y <= 2 or (0 <= x <= 3 and 0 <= y <= 5):
        return 1
    else:
        return 0
```

The logic flow of the grid initialisation is then:

<img src="./report_diagrams/init_grid.png" width=500x>

where 3 method set up the grid:

```python 
def place_point()
```

which creates a particle object, gives it basic parameters and adds it to the particle list of the main class

```python 
def add_boundaries()
```

which pads the grid outside of range [x_min, x_max] and calls `place_point` to add boundaries as bound particles

```python 
def initialise grid(f(x,y))
```

which loops over all coordinates and calls `place_point` where f(x, y) is true. Once this is done it calls `add_boundaries` and checks for duplicates. 


## Particle Leakage Control

Particle leakage in the simulation was identified as one of the main challenges of this sph simulation. Two main components were added to prevent from leakage of the system: 

### Lennard-Jones potential Boundary Forces

LJ forces were added to the calculations. This implementation adds an acceleration component to particles that are within desired distance to the wall to push them back into the domain. 

The desired distance is computed by ratio:

$$
{q}_{ref} = \frac{{d}_{ref}}{{r}_{wall-normal}}
$$

where $d_{ref}$ is a set fraction of the $dx$ set by the user, and ${r}_{wall-normal}$ is the perpendicular distance between the particle and the wall.If the ratio is larger than one, the LJ force are added to the acceleration of the particle.

The acceleration component is:

$$
{a}_{wall-normal} = \frac{{P}_{ref}({q}_{ref}^{4} - {q}_{ref}^{2})}{{r}_{wall-normal}{\rho}_{i}}
$$

and is added to the acceleration of the particle to push it back into the domain.

This boundary control was extensively tested and proved to work beyond expectations


### Leakage Damage Control

This code was implemented early on the code and were used as a initial control for leakages. It consists of checking wether a particle: - has neighbours, and -is within the grid domain. If the particle does not go through this validation it is earased for the domain particle list. 


## Particle Object 

All of the particle information is save in a particle object such as any property of the particle can be called from the instance of the main sph simulator. 

The particle properties are:

``` python
self.id = next(self._ids)
self.main_data = main_data
self.x = np.array(x)
self.x_temp = np.array(x)
self.v = np.zeros(2)
self.v_temp = np.zeros(2)
self.a = np.zeros(2)
self.D = 0
self.rho = 0.0
self.rho_temp = 0.0
self.P = 0.0
self.m = 0.0
self.bound = None
self.adj = []
```

## Front End

Once the code was up and running for all 3 modules, `sph_fe`, `sph_ie` and `sph_ap`, a front end was added to the code to ensure easy readability and use of the SPH simulator.


## Saving

When the system is initialised a config.txt file is saved which holds all the system parameters the save was run for. Then a csv file is opened which is appended to at a user specified frequency. The csv is set up into a readable column format, which allows for easy reading by the pandas library. This means if the user decided to stop the simulation early all data up to that point is still saved, and with the possible extension of checkpointing the config file could be used to re-setup the system. This would give the user large felexibility with our code.


## Modules Summary

Three modules were developed to compute SPH simulation. The table below summarizes their functionality:

| |Forward Euler| Predictor-Corrector| Find Neighbour Opt| Artificial Pressure| LJ Boundary Forces | Leak Control| Front End |
|--------| -------------| --------| -------| -------| -------| ------- | -------|
| `sph_fe`| x| | x| | x| x| x|
| `sph_ie`| | x| | | x| x| x|
| `sph_ap`| | x| | x| x| x| x|




# <span style="color:blue"> 2. Demonstration of Functionality </span>

The .csv files and animations of the simulation can be found in the file /examples of the github repository.

## Forward Euler Method

The best example of `sph_fe` is FE: 0.2 dx t=30s

## Predictor Corrector Method

The best example of `sph_ie` is IE: 0.2 dx t=11s

## Predictor Corrector and Artificial Pressure Method

The best example of `sph_ap` is AP: 0.2 dx t=6s

## Comparison

Comparing the flows obtained from using the Forward Euler method and the Predictor Corrector with additional artificial pressure, it seems that the artificial pressure introduced does in fact reduce tensile instability in the particles. Instead of flying off to the upward boundaries, they seem to be more contained with its neighbours with a local attraction.



# <span style="color:blue"> 3. Sustainability  </span>


The SPH simulator models a very physical system where all the values printed have to be physically feasible. In order to check for sustainability pytests and feasibility studies were performed

## Speed Sustainability

A pytest was written to check through the printed results. This test checks whether the velocity of the particles remains below the speed of sound in water. This is particurlarly important at boundary conditions, where the non-bound particles are given a larger accelartion whih might result in odd velocity. 

The test performed positively on all the data saved from the simulation.

## Density Sustainability

Density is another parameter that must remain within physical bounds.

Two tests were performed to ensure the data was consistent throughout the results of the simulations. The first one is a pytest that checks that no value is zero or negative. 

The second test ensures that the density does not dramatically vary away from the original density $\rho$, which in the test examples is 1000 $\frac{kg}{{m}^{3}}$. The average density, amx velocity and minimum velocity were plotted against time to visualise the changes, and ensure that at no point the density would vary by more than 1.5 times. 

<img src="./report_diagrams/den_15_comp.png" width=500x>

Here it can be seen that for a run of 30 seconds, the density is never reaching an unphysical value. There are two spikes in the data, at time 4s and 12s, and looking at the run of the simulation used here this happens when a particle leaks out of the system and is subsequantly deleted. 




## Overlapping Particles

In order to check that, a pytest was written. There is also a assert while initialising the grid.


## Asserting Physical Properties

These are the three main physical tests that were performed for the SPH simulations. On top of this, more physical properties are checked and asserted for through the codes, to ensure the simulation does not continue without analysis of the data. 

For example: 

```python
assert np.unique(tmp, axis=0).shape[0] == len(tmp), \
            'there is a duplicate point'
```
to check for overlapping particles at grid initialisation

```python
assert ((p_j_rho > 0).all()), "density must be always positive"
```

to check that $\rho$ is positive at the density smoothing. 


## Smoothing factor and Speed of Sound Study


The SPh simulations are simplified for the benefit of the computational efficiency. Two arbitrary constants in the simulationa are the smoothing length h, and the speed of sound in water c0.


### a. Speed of Sound Findings

c0 is a constant set to 20 ${ms}^{-1}$, however in real life c0 in water is approximately 1500 ${ms}^{-1}$. 

Different values of c0 were rune on `sph_fe` for small values of `t_final` and `dx` = 0.8.

a. It was found that the simulation took longer to run as the choice of c0 is increased

b. Four particles were considred, and the effect of c0 values on density and pressure were investigated. 

c. As c0 is increased from 20 to 200m the values for density decreased.

d. As c0 is increased from 20 to 200m, a higher variations in the pressure values were seen. 

This is exemplified below: 

<img src="./report_diagrams/c0_density.png" width=500x>

<img src="./report_diagrams/c0_pressure.png" width=500x>


### a. h-factor Findings


h is calculated arbitrarily and is proportional to $dx$:
$$
h = 1.3dx
$$

a. It was observed thar as the values of the h factor were reduced from 1.3 to 0.7, the simulation ran fasted. However when the value was increased to 1.5, the simulation was subsequently slower.

b The effect of change in h was again observed for two test case particles' pressure and density throughout the run.

c. As teh values of the h-factor decreases, lower values of density and pressure were observed. For lower values of h-factor the pressure and density were more unstable. 

<img src="./report_diagrams/h_density.png" width=500x>

<img src="./report_diagrams/h_pressure.png" width=500x>



# <span style="color:blue"> 4. Tracking the Wave Crest Position </span>


The function `peak(file_name, wallpos, mv_avN=1)` was implemented in order to  track the crest of peak  and return its  height and  x-position. It also returns the time when the wave sloashing interval (defined when xcoord >= 0.99wallpos). 

It works by splitting the data generated by the simulation into timeframes (removing the boundary particles) and for each timeframe defines the peak position as the x-coordinate of the particle with the highest height. This definition of the peak position can result in some noise as it does not take into consideration particles that are splashing. To deal with this issue, the user can choose to do a moving avarage to the height and x-coordinate values of the N nearest neighbours (nearest in terms of their x-coordinates) where, the input parameter 'mv_avN' defines N nearest neighbours.

Below is the animation of the data data we input
             
Below is our analysis of the data


`Sloashing interval (in s) is 3.385864
Simulated wave speed (in ms-1) is 5.906911
Analytical wave speed (in ms-1) is 4.902499`


<img src="./report_diagrams/crest1.png" width=500x>


In the first plot it can be seen that as the wave travels the wave is losing energy as its amplitude deacreases. However, the velocity stays the same (it hits the walls at constant time intervals of 3.5s where it splashes, this is also evident in the second plot where the crest takes a constant 3.5s to travel from one wall to another). This result was in agreement with the analytical result for velocity (where v depends on the average initial depth of water) since the simulation was run in a container that is not leaking. 


## Results from full simulation

The wave crest was computed for the full `sph_fe` dx = 0.2 for 30 seconds simulation. The results can be seen below

This is the test that goes below,



<img src="./report_diagrams/Peak_Analysis1.png" width=500x>

<img src="./report_diagrams/Peak_Analysis2.png" width=500x>



## Comparison between SPH simulator and shallow water velocities

Comparing simulated results to the analytical, we see that the simulated speed is an over estimate. We also observed that increasing the number of particles in the system deacreased the wave velocity (when we run the above simulation with dx = 0.2 we got tslosh = 3.67 and v = 5.61). 

The analytical velocity result comes from solving the wave equation, however, we are not in the continuum limit (this is probably the source of the discrepacy) and possibly as N rises V_simulation will converge to that of the analytical.

## Convergence Plot

<img src="./report_diagrams/convergence.png" width=700x>

In the above plot, we can see that indeed as we are moving towards the continuum limit the simulation velocities converges to that of the analytic wave velocity coming from the wave equation.





# <span style="color:blue"> 5. Artificial Pressure </span>

The artificial pressure was added to the SPH simulator. The smoothing function were derived from research, as follow. q

- The Artificial pressure term introduced by J. J Monaghan was implemented to improve the simulation results and asthetics as follows:


- Applying the artificial pressure term to the SPH acceleration of a particle 'a' gives;


$$a_i = \sum_{j=0}^{N} m_j \left(\frac{P_i}{\rho_i^2} + \frac{P_j}{\rho_j^2}\right) \frac{dW}{dr} e_{ij} +  \mu \sum_{j=0}^{N} m_j \left(\frac{1}{\rho_i^2} + \frac{1}{\rho_j^2}\right) \frac{dW}{dr} \frac{V_i}{\left|r_{ij}\right|} + \sum_{j=0}^{N} Rf_{ij}^n + g $$


where $r > 0$,  $R = f(P,\rho)$


$$R = R_i + R_j,   \in = 0.2 $$

$$R_i = \begin{cases}\in \frac{\left|P_{i}\right|}{\rho_{i}^2} , & \text{if $P_i<0$}.\\\\
0.01\left(\frac{P_i}{\rho_i^2} + \frac{P_j}{\rho_j^2}\right) , & \text{if $otherwise$}.\end{cases}$$


To get $R_j$ we replace $i$ with $j$ in the equation above


$$W_{i,j}^n = \frac{10}{7\pi h^2}\begin{cases} \left(1 - \frac{3}{2}q^2 + \frac{3}{4}q^3\right)^n, & \text{if $0<=q<=1$}.\\\\
\frac{1}{4}\left(2 - q\right)^{3n} , & \text{if $1<=q<=2$}, \\\\
0, & \text{if $q>2$}.\end{cases}$$


$$f_{ij}^n = \frac{dW_{ij}^n}{dr}$$


$$\frac{dW_{ij}^n}{dr} = \begin{cases} \frac{40}{7\pi h^3} \left(-3q + \frac{9}{4}q\right) \left(1 - \frac{3}{2}q^2 + \frac{3}{4}q^3\right)^3, & \text{if $0<=q<=1$}.\\\\
\frac{-30}{7\pi h^3}\left(2 - q\right)^{11} , & \text{if $1<=q<=2$}, \\\\
0, & \text{if $q>2$}.\end{cases}$$

# <span style="color:blue"> 6. Further Improvements</span>

For the development of this SPH simulation ran we had a limited amount of time. The simulator modules are consitant and sustainable, hence further research and improvements should be simple to do. In particluar further research on the results is key as the lond run times of singificant resolution simulations meant we did not have enough time to inspect the results very much.


## Convergence Test
Convergence test should be performed as part of the sustainabilty of the code and the results. There are two possible ways we could implement this:

1) Compare the wave crest time development to those expected from the simple wave equation. To do this one could either run a very wide simulation so the crest can be analysed both before it reflects off a boundary or the time period of the crest moving across the domain could be checked both for consistency and compared to the wave equations predictions. By varying dx and looking at the errors with respect to these criteria at a specified time should give a reasonable convergence plot.

2) Check the consistency of known conversion laws, for example conservation of energy. The possiblilty of particles leaking and the impact of the boundary Lenard-Jones potential on this would need to be carefully considered before an analysis of this type could be considered.

## Further Optimisation

#### Implementing checkpointing (~ 2hr)
We already made the system save a configuration file of the system object instances values at the start of the simulaiton. Using this with any of the time snaps saved (in the csv file) one could load the system to futher simulate. This would likely take the form of an external funciton that would take the config, data file names and return a instance of the system class setup with the particles and system parameters ready to be run.

#### Optimisation (~5 hr)
Currently we have only done the half neighbours optimisation for Forward Euler, this should be added to the improved euler and artifical pressure versions of the code need to be updated with this. There are also other optimisations that could be implemented, these include:

- Completely ignoring boundary particles with no non-boundary particle neighbours as the relative positions and velocities can't have changed (all boundary particles are fixed) and hence they can be ignored. These include all of the boundary partilces in the upper half of the domain, significantly reducing the number of particles to iterate over, so would give some significant improvements
- Not finding acceleration for any boundary particles, as they always remain fixed. This would reduce the number of calculations for these particles so provided a minor improvement
- locally store relative particle positions as these are being calculated once to find neighbours and once to find the kernal funciton value. Again this reduces number of calculations for these particles so provided a minor improvement.
- Beyond this running a profiler to further improve the code could be done

All these improve run time but none to the extent of the half neighbour method so were given a lower priortiy and not finished within the deadline.

#### Further analysis of results (~3hr)

Taking the data and tracing the path and property change of single particles for different system parameters would provided a good insight into the impact these parameters have on the simulation. Combining this with the overall trends of the system we could look with more detail into how $C_0$ and $h_fac$ impact the results and combination with previously described convergence tests could show which settings are most realistic.